# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [43]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scikit-learn in c:\users\javi\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (1.6.1)




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Javi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [44]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [45]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [46]:
spaceship.shape

(8693, 14)

In [47]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [48]:
spaceship.isnull()

PassengerId  HomePlanet  CryoSleep  Cabin  Destination    Age    VIP  \
0           False       False      False  False        False  False  False   
1           False       False      False  False        False  False  False   
2           False       False      False  False        False  False  False   
3           False       False      False  False        False  False  False   
4           False       False      False  False        False  False  False   
...           ...         ...        ...    ...          ...    ...    ...   
8688        False       False      False  False        False  False  False   
8689        False       False      False  False        False  False  False   
8690        False       False      False  False        False  False  False   
8691        False       False      False  False        False  False  False   
8692        False       False      False  False        False  False  False   

      RoomService  FoodCourt  ShoppingMall    Spa  VRDeck   Name  Transported  
0           False      False         False  False   False  False        False  
1           False      False         False  False   False  False        False  
2           False      False         False  False   False  False        False  
3           False      False         False  False   False  False        False  
4           False      False         False  False   False  False        False  
...           ...        ...           ...    ...     ...    ...          ...  
8688        False      False         False  False   False  False        False  
8689        False      False         False  False   False  False        False  
8690        False      False         False  False   False  False        False  
8691        False      False         False  False   False  False        False  
8692        False      False         False  False   False  False        False  

[8693 rows x 14 columns]

In [49]:
spaceship.isnull().sum()


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [50]:
spaceship.dropna()

PassengerId HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
0        0001_01     Europa     False     B/0/P    TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F/0/S    TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A/0/S    TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A/0/S    TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F/1/S    TRAPPIST-1e  16.0  False   
...          ...        ...       ...       ...            ...   ...    ...   
8688     9276_01     Europa     False    A/98/P    55 Cancri e  41.0   True   
8689     9278_01      Earth      True  G/1499/S  PSO J318.5-22  18.0  False   
8690     9279_01      Earth     False  G/1500/S    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa     False   E/608/S    55 Cancri e  32.0  False   
8692     9280_02     Europa     False   E/608/S    TRAPPIST-1e  44.0  False   

      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0             0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1           109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2            43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3             0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4           303.0       70.0         151.0   565.0     2.0  Willy Santantines   
...           ...        ...           ...     ...     ...                ...   
8688          0.0     6819.0           0.0  1643.0    74.0  Gravior Noxnuther   
8689          0.0        0.0           0.0     0.0     0.0    Kurta Mondalley   
8690          0.0        0.0        1872.0     1.0     0.0       Fayey Connon   
8691          0.0     1049.0           0.0   353.0  3235.0   Celeon Hontichre   
8692        126.0     4688.0           0.0     0.0    12.0   Propsh Hontichre   

      Transported  
0           False  
1            True  
2           False  
3           False  
4            True  
...           ...  
8688        False  
8689        False  
8690         True  
8691        False  
8692         True  

[6606 rows x 14 columns]

In [51]:
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else x)
spaceship.head()

PassengerId HomePlanet CryoSleep Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False     B  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False     F  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False     A  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False     A  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False     F  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

In [52]:
spaceship.fillna(0, inplace=True)

In [53]:
spaceship = spaceship.drop(columns = ["PassengerId",  "Name"])


In [54]:
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Transported'], drop_first=True)
spaceship.head()

Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0  39.0          0.0        0.0           0.0     0.0     0.0   
1  24.0        109.0        9.0          25.0   549.0    44.0   
2  58.0         43.0     3576.0           0.0  6715.0    49.0   
3  33.0          0.0     1283.0         371.0  3329.0   193.0   
4  16.0        303.0       70.0         151.0   565.0     2.0   

   HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  ...  \
0             False               True            False           False  ...   
1              True              False            False           False  ...   
2             False               True            False           False  ...   
3             False               True            False           False  ...   
4              True              False            False           False  ...   

   Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Destination_55 Cancri e  \
0    False    False    False    False    False                    False   
1    False    False     True    False    False                    False   
2    False    False    False    False    False                    False   
3    False    False    False    False    False                    False   
4    False    False     True    False    False                    False   

   Destination_PSO J318.5-22  Destination_TRAPPIST-1e  VIP_True  \
0                      False                     True     False   
1                      False                     True     False   
2                      False                     True      True   
3                      False                     True     False   
4                      False                     True     False   

   Transported_True  
0             False  
1              True  
2             False  
3             False  
4              True  

[5 rows x 23 columns]

**Perform Train Test Split**

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

In [55]:
features = spaceship.drop(columns = ["Transported_True"])
target = spaceship["Transported_True"]

In [56]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [57]:
normalizer = MinMaxScaler()

In [58]:
normalizer.fit(X_train)

MinMaxScaler()

In [59]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [60]:
X_train_norm

array([[6.83544304e-01, 0.00000000e+00, 2.01637629e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.53164557e-01, 0.00000000e+00, 7.21422645e-04, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.44303797e-01, 1.27102673e-01, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [3.67088608e-01, 3.65045020e-02, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.69620253e-01, 0.00000000e+00, 3.24640190e-04, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [61]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck  \
0  0.683544     0.000000   0.020164      0.000000  0.820482  0.115982   
1  0.253165     0.000000   0.000721      0.000043  0.037476  0.000000   
2  0.544304     0.127103   0.000000      0.002001  0.001561  0.000000   
3  0.303797     0.012913   0.000000      0.020262  0.097459  0.002196   
4  0.316456     0.000000   0.000000      0.000000  0.000000  0.000000   

   HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  ...  \
0               0.0                1.0              0.0             0.0  ...   
1               1.0                0.0              0.0             0.0  ...   
2               0.0                0.0              1.0             0.0  ...   
3               1.0                0.0              0.0             0.0  ...   
4               0.0                1.0              0.0             1.0  ...   

   Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  \
0      1.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0      0.0      1.0      0.0      0.0   
2      0.0      0.0      0.0      1.0      0.0      0.0   
3      0.0      0.0      0.0      1.0      0.0      0.0   
4      1.0      0.0      0.0      0.0      0.0      0.0   

   Destination_55 Cancri e  Destination_PSO J318.5-22  \
0                      1.0                        0.0   
1                      0.0                        0.0   
2                      0.0                        0.0   
3                      0.0                        0.0   
4                      1.0                        0.0   

   Destination_TRAPPIST-1e  VIP_True  
0                      0.0       0.0  
1                      1.0       0.0  
2                      1.0       0.0  
3                      1.0       0.0  
4                      0.0       0.0  

[5 rows x 22 columns]

In [62]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck  \
0  0.202532     0.000000   0.000000      0.026520  0.002154  0.008660   
1  0.025316     0.000000   0.000000      0.000000  0.000000  0.000000   
2  0.392405     0.036086   0.000036      0.012813  0.003231  0.000166   
3  0.177215     0.045578   0.000000      0.000170  0.000000  0.000000   
4  0.379747     0.000000   0.000000      0.000000  0.000000  0.000000   

   HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  ...  \
0               1.0                0.0              0.0             0.0  ...   
1               1.0                0.0              0.0             0.0  ...   
2               1.0                0.0              0.0             0.0  ...   
3               1.0                0.0              0.0             0.0  ...   
4               0.0                1.0              0.0             1.0  ...   

   Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  \
0      0.0      0.0      0.0      1.0      0.0      0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0      0.0      0.0      0.0      1.0      0.0   
3      0.0      0.0      0.0      1.0      0.0      0.0   
4      0.0      0.0      0.0      0.0      0.0      0.0   

   Destination_55 Cancri e  Destination_PSO J318.5-22  \
0                      1.0                        0.0   
1                      0.0                        0.0   
2                      0.0                        0.0   
3                      1.0                        0.0   
4                      1.0                        0.0   

   Destination_TRAPPIST-1e  VIP_True  
0                      0.0       0.0  
1                      0.0       0.0  
2                      1.0       0.0  
3                      0.0       0.0  
4                      0.0       1.0  

[5 rows x 22 columns]

- Bagging and Pasting

In [63]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [64]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [66]:
pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.2817336299531379
RMSE 0.1402091407461748
R2 score 0.4391320934691254


- Random Forests

In [69]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [70]:
forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=20)

In [72]:
pred = forest.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", forest.score(X_test_norm, y_test))

MAE 0.27620910746396704
RMSE 0.1490066501915471
R2 score 0.40394008901740297


- Gradient Boosting

In [26]:
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [27]:
gb_reg.fit(X_train_norm, y_train)

GradientBoostingRegressor(max_depth=20)

In [67]:
pred = gb_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", gb_reg.score(X_test_norm, y_test))

MAE 0.27740889599060337
RMSE 0.1942674333125047
R2 score 0.2228868385524042


- Adaptive Boosting

In [29]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [30]:
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [68]:
pred = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.26001793332195333
RMSE 0.18822916109420434
R2 score 0.24704127727243508


Which model is the best and why?

In [25]:
#el mejor modelo es el Bagging porque tiene el menor error cuadratico medio y el mayor R2 score.